In [2]:
from bs4 import BeautifulSoup as soup
import requests

In [3]:
#page = soup(requests.get('http://www.imsdb.com/scripts/American-Sniper.html').text)
page = soup(requests.get('http://www.imsdb.com/transcripts/Futurama-Space-Pilot-3000.html').text)

/usr/local/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/local/Cellar/python/2.7.12_2/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [4]:
all_bs = page.find_all('b')

In [11]:
all_bs

[<b>                                        FUTURAMA\n</b>,
 <b>                                   "SPACE PILOT 3000"\n</b>,
 <b>                \n</b>,
 <b> \n</b>,
 <b>               \n</b>,
 <b>               \n</b>,
 <b>                                     MAN\n</b>,
 <b> \n</b>,
 <b>               \n</b>,
 <b> \n</b>,
 <b>               \n</b>,
 <b>                                     FRY\n</b>,
 <b>               \n</b>,
 <b>                                     KID\n</b>,
 <b>               \n</b>,
 <b> \n</b>,
 <b>               \n</b>,
 <b>                                     PANUCCI\n</b>,
 <b> \n</b>,
 <b>                         \n</b>,
 <b>               \n</b>,
 <b>               \n</b>,
 <b> \n</b>,
 <b>               \n</b>,
 <b>                                     FRY\n</b>,
 <b>               \n</b>,
 <b>                                     MICHELLE\n</b>,
 <b> \n</b>,
 <b>               \n</b>,
 <b> \n</b>,
 <b>               \n</b>,
 <b>               \n</b>,
 <b>   

In [47]:
p = all_bs[5]

In [52]:
p.next_sibling.split('\r\n')[0]

[u'                 Fucking hot box.',
 u'',
 u'    GOAT (24, Arkansas Marine) lies beside him, woodsy and',
 u'    outspoken, watching dirt-devils swirl in the street.',
 u'',
 u'']

In [78]:
re.sub(r'\([^)]*\)', '','(askfjd)')

''

In [84]:
x = 'RTMa'
x.isupper()

False

In [24]:
def faturama_extractor(url):
    page = soup(requests.get(url).text)
    all_bs = page.find_all('b')
    num_match = re.compile("\d+\r\n")
    
    dialogs = {}
    scene = 0
    for bs in all_bs:
        # detect scene change
        if len(num_match.split(bs.text)) > 1:
            scene += 1
            continue
        if scene > -1:
            try:
                spaces = len(bs.text) - len(bs.text.lstrip())
                if spaces > 15:
                    siblings = bs.next_sibling.split('\r\n')
                    if len(siblings) > 0:
                        siblings = [re.sub(r'\([^)]*\)', '', s).strip() for s in siblings]
                        #print siblings
                        siblings = [s for s in siblings if len(s.lstrip()) > 0]
                        #print siblings
                        if scene not in dialogs:
                            dialogs[scene] = []
                        #print siblings
                        dlg = []
                        #print siblings
                        for sib in siblings:
                            words = sib.split(' ')
                            if (words[0].isupper() and words[1].isupper()) or (len(words[0]) > 1 and words[0].isupper()):
                                break
                            dlg.append(sib)
                        dlg = ' '.join(dlg)
                        if '[' in dlg and ']' in dlg:
                            if 'Cut to' in dlg:
                                scene +=1
                                continue
                        else:
                            dlg = ' '.join(dlg.split())
                            dialogs[scene].append((' '.join(bs.text.split()),dlg))
            except:
                continue
    return dialogs
            

In [25]:
dialogs = faturama_extractor('http://www.imsdb.com/transcripts/Futurama-Space-Pilot-3000.html')

In [30]:
import unicodecsv as csv
def dialog_saver(dialogs,filename,season=0,episode=0):
    with open(filename,'w') as fp:
        writer = csv.writer(fp, delimiter=',',encoding='utf-8')
        writer.writerow(['Season','Episode','Scene','Character','Line'])
        for scene,dlgs in dialogs.iteritems():
            for dtuple in dlgs:
                character = dtuple[0]
                line = dtuple[1]
                if len(character) > 0 or len(line) > 0:
                    writer.writerow([season, episode, scene, character, line])

In [35]:
links = {'1':['http://www.imsdb.com/transcripts/Futurama-Space-Pilot-3000.html',
             'http://www.imsdb.com/transcripts/Futurama-The-Series-Has-Landed.html',
             'http://www.imsdb.com/transcripts/Futurama-I,-Roommate.html',
             "http://www.imsdb.com/transcripts/Futurama-Love's-Labours-Lost-In-Space.html",
             "http://www.imsdb.com/transcripts/Futurama-Fear-Of-A-Bot-Planet.html",
             "http://www.imsdb.com/transcripts/Futurama-A-Fishful-Of-Dollars.html",
             "http://www.imsdb.com/transcripts/Futurama-My-Three-Suns.html",
             "http://www.imsdb.com/transcripts/Futurama-A-Big-Piece-Of-Garbage.html",
             "http://www.imsdb.com/transcripts/Futurama-Hell-Is-Other-Robots.html"],
         '2':["http://www.imsdb.com/transcripts/Futurama-A-Flight-To-Remember.html",
             "http://www.imsdb.com/transcripts/Futurama-Mars-University.html",
             "http://www.imsdb.com/transcripts/Futurama-When-Aliens-Attack.html",
             "http://www.imsdb.com/transcripts/Futurama-Fry-And-The-Slurm-Factory.html",
             "http://www.imsdb.com/transcripts/Futurama-I-Second-That-Emotion.html",
             "http://www.imsdb.com/transcripts/Futurama-Brannigan,-Begin-Again.html",
             "http://www.imsdb.com/transcripts/Futurama-A-Head-In-The-Polls.html",
             "http://www.imsdb.com/transcripts/Futurama-Xmas-Story.html",
             "http://www.imsdb.com/transcripts/Futurama-Why-Must-I-Be-A-Crustacean-In-Love.html",
             "http://www.imsdb.com/transcripts/Futurama-Put-Your-Head-On-My-Shoulder.html",
             "http://www.imsdb.com/transcripts/Futurama-The-Lesser-Of-Two-Evils.html",
             "http://www.imsdb.com/transcripts/Futurama-Raging-Bender.html",
             "http://www.imsdb.com/transcripts/Futurama-A-Bicyclops-Built-For-Two.html",
             "http://www.imsdb.com/transcripts/Futurama-How-Hermes-Requisitioned-His-Groove-Back.html",
             "http://www.imsdb.com/transcripts/Futurama-A-Clone-Of-My-Own.html",
             "http://www.imsdb.com/transcripts/Futurama-The-Deep-South.html",
             "http://www.imsdb.com/transcripts/Futurama-Bender-Gets-Made.html",
             "http://www.imsdb.com/transcripts/Futurama-The-Problem-With-Popplers.html",
             "http://www.imsdb.com/transcripts/Futurama-Mother's-Day.html",
             "http://www.imsdb.com/transcripts/Futurama-Anthlogy-Of-Interest-I.html"],
         '3':["http://www.imsdb.com/transcripts/Futurama-The-Honking.html",
             "http://www.imsdb.com/transcripts/Futurama-War-Is-The-H-Word.html",
             "http://www.imsdb.com/transcripts/Futurama-The-Cryonic-Woman.html",
             "http://www.imsdb.com/transcripts/Futurama-Parasites-Lost.html",
             "http://www.imsdb.com/transcripts/Futurama-Amazon-Women-In-The-Mood.html",
             "http://www.imsdb.com/TV%20Transcripts/Futurama%20-%20Bendless%20Love%20Script.html",
             "http://www.imsdb.com/transcripts/Futurama-The-Day-The-Earth-Stood-Stupid.html",
             "http://www.imsdb.com/transcripts/Futurama-That's-Lobstertainment!.html",
             "http://www.imsdb.com/transcripts/Futurama-The-Birdbot-Of-IceCatraz.html",
             "http://www.imsdb.com/transcripts/Futurama-Luck-Of-The-Fryrish.html",
             "http://www.imsdb.com/transcripts/Futurama-The-Cyber-House-Rules.html",
             "http://www.imsdb.com/transcripts/Futurama-Insane-In-The-Mainframe.html",
             "http://www.imsdb.com/transcripts/Futurama-Bendin'-In-The-Wind.html",
             "http://www.imsdb.com/transcripts/Futurama-Time-Keeps-On-Slipping.html",
             "http://www.imsdb.com/transcripts/Futurama-I-Dated-A-Robot.html"],
         '4':["http://www.imsdb.com/transcripts/Futurama-Roswell-That-Ends-Well.html",
             "http://www.imsdb.com/transcripts/Futurama-A-Tale-Of-Two-Santas.html",
             "http://www.imsdb.com/transcripts/Futurama-Anthology-Of-Interest-II.html",
             "http://www.imsdb.com/transcripts/Futurama-Love-And-Rocket.html",
             "http://www.imsdb.com/transcripts/Futurama-Leela's-Homeworld.html",
             "http://www.imsdb.com/transcripts/Futurama-Where-The-Buggalo-Roam.html",
             "http://www.imsdb.com/transcripts/Futurama-A-Pharaoh-To-Remember.html",
             "http://www.imsdb.com/transcripts/Futurama-Godfellas.html",
             "http://www.imsdb.com/transcripts/Futurama-Future-Stock.html",
             "http://www.imsdb.com/transcripts/Futurama-A-Leela-Of-Her-Own.html",
             "http://www.imsdb.com/transcripts/Futurama-30-Percent-Iron-Chef.html",
             "http://www.imsdb.com/transcripts/Futurama-Where-No-Fan-Has-Gone-Before.html"],
         '5':["http://www.imsdb.com/transcripts/Futurama-Crimes-Of-The-Hot.html",
             "http://www.imsdb.com/transcripts/Futurama-Jurassic-Bark.html",
             "http://www.imsdb.com/transcripts/Futurama-The-Route-Of-All-Evil.html",
             "http://www.imsdb.com/transcripts/Futurama-A-Taste-Of-Freedom.html",
             "http://www.imsdb.com/transcripts/Futurama-Kif-Gets-Knocked-Up-A-Notch.html",
             "http://www.imsdb.com/transcripts/Futurama-Less-Than-Hero.html",
             "http://www.imsdb.com/transcripts/Futurama-Teenage-Mutant-Leela's-Hurdles.html",
             "http://www.imsdb.com/transcripts/Futurama-The-Why-Of-Fry.html",
             "http://www.imsdb.com/transcripts/Futurama-The-Sting.html",
             "http://www.imsdb.com/transcripts/Futurama-The-Farnsworth-Parabox.html",
             "http://www.imsdb.com/transcripts/Futurama-Three-Hundred-Big-Boys.html",
             "http://www.imsdb.com/transcripts/Futurama-Spanish-Fry.html",
             "http://www.imsdb.com/transcripts/Futurama-Bend-Her.html",
             "http://www.imsdb.com/transcripts/Futurama-Obsoletely-Fabulous.html",
             "http://www.imsdb.com/transcripts/Futurama-Bender-Should-Not-Be-Allowed-On-Tv.html",
             "http://www.imsdb.com/transcripts/Futurama-The-Devil's-Hands-Are-Idle-Playthings.html"]
         }

In [36]:
for si,season in enumerate(links):
    eps = links[season]
    if season in ["1","2"]:
        continue
    for i,ep in enumerate(eps):
        dialogs = faturama_extractor(ep)
        dialog_saver(dialogs, 'data/faturama/season_{}_ep_{}.csv'.format(season,i), season=season, episode=i)

In [6]:
import re

In [7]:
num_match = re.compile("\d+\r\n")

In [18]:
if num_match.match(all_bs[1].text):
    'yep'
else:
    'nope'

In [24]:
x = all_bs[1].text

In [43]:
num_match.split(x)

[u'1   OVER BLACK                                                  ', u'']

In [41]:
x.split('\r\n')

[u'1   OVER BLACK                                                  1\r\n']

In [35]:
num_match.match(str(x))

In [36]:
regex = r"\d+\\r\\n"

test_str = "1   OVER BLACK  ldskjfdslfkjdsf                                                 1\r\n"

matches = re.finditer(regex, test_str)

In [37]:
matches

<callable-iterator at 0x104d17050>

In [38]:
for m in matches:
    print m